Rudolph City Visit Optimization Kaggle Contest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("cities.csv")
print(data.shape)
print(data.head(10))

(197769, 3)
   CityId            X            Y
0       0   316.836739  2202.340707
1       1  4377.405972   336.602082
2       2  3454.158198  2820.053011
3       3  4688.099298  2935.898056
4       4  1010.696952  3236.750989
5       5  2474.230877  1435.514651
6       6  1029.277795  2721.800952
7       7  3408.887685   199.585793
8       8  1304.006125  2657.427246
9       9  4211.525725  2294.595208


New function to figure out numbers are prime

In [ ]:
def is_prime(n):
    if n > 2:
        i = 2
        while i ** 2 <= n:
            if n % i:
                i += 1
            else:
                return False
    elif n != 2:
        return False
    return True

In [ ]:
data['Prime'] = data.CityId.apply(is_prime)
print(data.head(5))

In [ ]:
def distance_traveled(data,route):
    previous_city = route[0]
    traveled_distance = 0
    penalty_distance = 0
    step = 1
    for city_number in route[1:]:
        next_city = city_number
        traveled_distance += np.sqrt((data.X[city_number] - data.X[previous_city])**2 + (data.Y[city_number] - data.Y[previous_city])**2) * (1 + 0.1 * (step % 10 ==0)*int(data.Prime[previous_city]))
        # For now no penalties involved
        step += 1
        previous_city = next_city
    return traveled_distance


In [10]:
route_basic = list(data.CityId[:].append(pd.Series([0])))
route_count = route_basic[:-1]

In [ ]:
distance_basic = distance_traveled(data,route_basic)

In [ ]:
print(distance_basic)

Distance function seems to be working (at least well enough for testing) 
Now try to optimize route a little bit more

Try sorting things a little bit: Start with just sorting by X and Y

In [ ]:
data_sortedX = data.sort_values(by=['X'])
data_sortedY = data.sort_values(by=['Y'])

In [ ]:
route_X_sorted = list(data_sortedX.CityId[:].append(pd.Series([0])))
route_Y_sorted = list(data_sortedY.CityId[:].append(pd.Series([0])))

In [ ]:
distance_X_sort = distance_traveled(data,route_X_sorted)
distance_Y_sort = distance_traveled(data,route_Y_sorted)
# probably have to rework this since 0 isn't first in list, but oh well

In [ ]:
print("Basic Order: ", distance_basic)
print("X Sort:      ",distance_X_sort)
print("Y Sort:      ",distance_Y_sort)

All of the numbers are in the top right quadrant (all positive)
Create a third category (distance from origin) and sort by that

In [ ]:
def distance_origin(X,Y):
    Z = np.sqrt(X**2 + Y**2)
    return Z

In [ ]:
data["Z"] = distance_origin(data.X,data.Y)

In [ ]:
data_sortedZ = data.sort_values(by=['Z'])
route_Z_sorted = list(data_sortedZ.CityId[:].append(pd.Series([0])))
distance_Z_sort = distance_traveled(data,route_Z_sorted)
print("Z sort: ", distance_Z_sort)

Make a Nearest Neighbors Table for each point (5 each?)

In [ ]:
NN_dist_table = np.ones((len(route_count),5))*200 
NN_city_table = np.zeros((len(route_count),5))

In [ ]:
distance = 0
for i in route_count:
    for j in route_count:
        distance = np.sqrt((data.X[i] - data.X[j])**2 + (data.Y[i] - data.Y[j])**2)
        if (distance > 0) and (distance < max(NN_dist_table[i,:])):
            index = np.argmax(NN_dist_table[i,:], axis = None)
            NN_dist_table[i,index] = distance
            NN_city_table[i,index] = j
    if i % 10 == 0:
        print(i)

In [ ]:
print(NN_city_table[0:3,:])

In [ ]:
nearest_5 = np.zeros((len(route_count),5))
xy = np.array([data.X.values,data.Y.values]).T
for i in route_count:
    current_xy = np.array([data.X[i],data.Y[i]])
    distances = np.sqrt(((xy - current_xy)**2).sum(-1))
    nearest_5[i,] = distances.argsort()[:5]
    